# Impact of COVID-19 on the U.S. Housing Market 🏠

## Introduction

Real estate and the housing market play an important role in the U.S. economy. The U.S. housing market accounts for a significant 17.5% of GDP (2020) and changes in the housing market can have broader effects on the economy. 

> The outbreak of COVID-19 in 2019 greatly affected the economy in the United States. In this project, we will analyse the impact of the pandemic on the U.S. housing market in particular between 2017-2022. These insights would be helpful to understand so that buyers / sellers, real estate brokerages, policy makers, and the government can make better decisions in the future.

We would like to understand market trends, price volatility, buying patterns, and changes in supply and demand across the country before and after the onset of the COVID-19 pandemic. We also want to learn if these effects were consistent across: low-priced and high-priced housing markets, different types of residential properties (condo, townhouse, multi-family, single-family homes etc.), and metro vs. non-metro areas. We would also like to understand if there is a correlation between the % population tested positive, mortality rate, and vaccination rates on the housing market at the county level. 

By Aparna Gopalakrishnan (agopala@umich.edu), Varshini Rana (varshini@umich.edu), and Ajay Prasad (ajaypras@umich.edu)

## Notebook setup

As a first step let's record and import all dependencies for this notebook. We've mostly used numpy, pandas, matplotlib, scipy, and statsmodels libraries.

In [ ]:
!pip install sodapy
!pip install statsmodels
!pip install plotly-geo==1.0.0
!pip install geopandas==0.3.0
!pip install pyshp==1.2.10
!pip install shapely
!pip install pandas==1.3.5

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 KB 53.3 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.9 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.7/888.7 KB 66.4 MB/s eta 0:00:00
  Attempting uninstall: geopandas
    Found existing installation: geopandas 0.11.1
    Not uninstalling geopandas at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'geopandas'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━

In [ ]:
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import scipy
import scipy.stats as st
import statsmodels
import statsmodels.api as sm
import plotly.figure_factory as ff
import plotly.express as px

print('Pandas:', pd.__version__)
print('Matplotlib: ', matplotlib.__version__)
print('Scipy: ', scipy.__version__)
print('Statsmodels: ', statsmodels.__version__)

Pandas: 1.2.5
Matplotlib:  3.5.3
Scipy:  1.9.1
Statsmodels:  0.13.2


## Loading the datasets

Now, let's load all the three datasets and print out the shape, datatypes of columns, and check out a few rows.

### 🏠 U.S. Housing Market Data (primary)

Our primary dataset is the U.S. Housing Market Data that will be collected from Redfin. Redfin is a U.S. based real estate brokerage firm with access to data and insights on listings across the country. We will employ aggregated time series data of housing prices at a county level from January 2012 through July 2022. Specifically, it contains historical data (by month and county) of housing inventory, sales, average / median price change (month-over-month / year-over-year) etc. and has approximately 700K records with 50+ attributes.

In [ ]:
# Primary tsv dataset
tsv_file = 'county_market_tracker.tsv'
 
# Reading the dataset tsv file through read_table()
df_housing = pd.read_table(tsv_file)

# Let's find out the shape of the three datasets
print("Shape of the dataset:{}".format(df_housing.shape))

# Let's check out the datatypes of the columns of the 2020 dataset
print(df_housing.dtypes)

# Let's check out the first few rows of the 2020 dataset
df_housing.head()

Shape of the dataset:(716333, 58)
period_begin                       object
period_end                         object
period_duration                     int64
region_type                        object
region_type_id                      int64
table_id                            int64
is_seasonally_adjusted             object
region                             object
city                              float64
state                              object
state_code                         object
property_type                      object
property_type_id                    int64
median_sale_price                 float64
median_sale_price_mom             float64
median_sale_price_yoy             float64
median_list_price                 float64
median_list_price_mom             float64
median_list_price_yoy             float64
median_ppsf                       float64
median_ppsf_mom                   float64
median_ppsf_yoy                   float64
median_list_ppsf                  float64


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2017-05-01,2017-05-31,30,county,5,2304,f,"Osage County, OK",NaN,Oklahoma,...,0.078285,0.23871,0.034484,0.088383,0.380952,-0.073593,0.180952,"Tulsa, OK",46140.0,2022-09-11 14:46:03
1,2022-01-01,2022-01-31,30,county,5,875,f,"Perry County, IN",NaN,Indiana,...,0.000000,NaN,NaN,NaN,0.000000,0.000000,NaN,Indiana nonmetropolitan area,NaN,2022-09-11 14:46:03
2,2017-07-01,2017-07-31,30,county,5,1591,f,"Walthall County, MS",NaN,Mississippi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mississippi nonmetropolitan area,NaN,2022-09-11 14:46:03
3,2013-06-01,2013-06-30,30,county,5,3218,f,"St. Croix County, WI",NaN,Wisconsin,...,0.031133,NaN,NaN,NaN,0.212121,-0.066725,0.126407,"Minneapolis, MN",33460.0,2022-09-11 14:46:03
4,2017-09-01,2017-09-30,30,county,5,3182,f,"Fond du Lac County, WI",NaN,Wisconsin,...,0.119048,0.10000,-0.030435,0.067742,0.000000,0.000000,0.000000,"Fond du Lac, WI",22540.0,2022-09-11 14:46:03


### 🦠 COVID-19 Cases and Deaths Data (secondary)

To analyse the impact of COVID-19 on housing prices, we will rely on the time-series COVID-19 cases data provided by the New York Times, which is an American daily newspaper. The data contains a daily number of confirmed cases and deaths in each county and state across the U.S since the beginning of the pandemic. More specifically, there is daily level data from 21st January 2020 through 8th September 2022. There is one CSV file per year. The Github location to access the datasets is here: https://github.com/nytimes/covid-19-data.


In [ ]:
# Secondary dataset urls
url_2020 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv"
url_2021 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv"
url_2022 = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2022.csv"

# Reading all the datasets
df_covid_2020 = pd.read_csv(url_2020)
df_covid_2021 = pd.read_csv(url_2021)
df_covid_2022 = pd.read_csv(url_2022)

# Let's find out the shape of the three datasets
print("Shape of the dataset:{}".format(df_covid_2020.shape))
print("Shape of the dataset:{}".format(df_covid_2021.shape))
print("Shape of the dataset:{}".format(df_covid_2022.shape))

# Let's check out the datatypes of the columns of the 2020 dataset
print(df_covid_2020.dtypes)

# Let's check out the first few rows of the 2020 dataset
df_covid_2020.head()

Shape of the dataset:(884737, 6)
Shape of the dataset:(1185373, 6)
Shape of the dataset:(914734, 6)
date       object
county     object
state      object
fips      float64
cases       int64
deaths    float64
dtype: object


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


### 💉 COVID-19 Vaccination Data (secondary)

To enrich the COVID-19 data set, we also plan to rely on the weekly vaccination data set provided by the Centers for Disease Control and Prevention (CDC) at a county level. The main feature in this data set is the percentage of the population who completed all the vaccine shots in each county and state across the U.S with additional features to distinguish between metro and non-metro counties. This feature can be further broken down by age bins. For example, % of users 12+ who completed the vaccine, % of users 65+ who completed the vaccine, etc. The data set starts from 13th December 2020 through 7th September 2022 in one file with 1.85M rows and 72 columns.

In [ ]:
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cdc.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("8xkx-amqh", limit=2000000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Let's find out the shape of the three datasets
print("Shape of the dataset:{}".format(results_df.shape))

# Let's check out the datatypes of the columns of this dataset
print(results_df.dtypes)

# Let's check out the first few rows of this dataset
results_df.head()

KeyboardInterrupt: 

In [ ]:
results_df.to_pickle('covid_vaccination_data.pickle')

In [ ]:
pip show pandas

Name: pandas
Version: 1.2.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: 
License: BSD
Location: /shared-libs/python3.9/py/lib/python3.9/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, datascience, geopandas, seaborn, statsmodels
Note: you may need to restart the kernel to use updated packages.


## Cleaning the dataset

### U.S. Housing Market Data (primary)

In [ ]:
# Convert date columns from object data type to datetime data type.
df_housing["period_begin"]=pd.to_datetime(df_housing["period_begin"])
df_housing["period_end"]=pd.to_datetime(df_housing["period_end"])

In [ ]:
# Filter the housing prices dataframe to include only data for years 2017 through 2022.
df_housing=df_housing.loc[df_housing["period_begin"]>="2017"].sort_values("period_begin")

In [ ]:
# Create a list of columns containing housing price details.
price_cols=list(df_housing.columns[13:-3])
# Replace missing values in price columns with medians according to state and property type.
for col in price_cols:
    df_housing[col].fillna(df_housing.groupby(["state", "property_type"])[col].transform("median"), 
    inplace=True)

In [ ]:
# Drop columns unnecessary for analysis.
df_housing.drop(["region_type", "region_type_id", "table_id", "is_seasonally_adjusted", "city", "last_updated"], 
axis=1, inplace=True)

In [ ]:
# Create a column containing the county name unassociated with state code.
df_housing["county"]=df_housing["region"].str.split(",").str[0]

In [ ]:
df_housing.head(2)

,period_begin,period_end,period_duration,region,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,county
383305,2017-01-01,2017-01-31,30,"Jefferson County, FL",Florida,FL,All Residential,-1,90000.0,-0.632653,...,0.00000,0.200077,0.000813,0.005734,0.400000,0.400000,0.400000,"Tallahassee, FL",45220.0,Jefferson County
471078,2017-01-01,2017-01-31,30,"Santa Rosa County, FL",Florida,FL,Single Family Residential,6,202670.0,0.011327,...,0.03818,0.211595,0.001102,0.007240,0.072072,0.033904,0.006498,"Pensacola, FL",37860.0,Santa Rosa County


In [ ]:
# remove median sale price 1
df_housing=df_housing.loc[~(df_housing["median_sale_price"]==1)]

# new feature
df_housing["sale_price_category"]=pd.cut(df_housing["median_sale_price"], \
                                         bins=[0, 50000, 100000, 200000, 300000, 500000, 1000000000],\
                                         labels=["Very low", "Low", "Average", "Above average", "High", "Very high"])


### COVID-19 Cases and Deaths Data (secondary)

In this section, let's clean and manipulate the COVID-19 cases and deaths dataset.

In [ ]:
# 1. There are three CSV files with COVID-19 data, one for each of the years 2020, 2021, and 2022.
# As a first step, these need to be concatenated together to form the complete dataset.

pdList = [df_covid_2020, df_covid_2021, df_covid_2022]
df_covid_combined = pd.concat(pdList)

# Let's print the shape of the combined dataset
print("Shape of the dataset:{}".format(df_covid_combined.shape))

# 2. There are some records where the patient’s county of residence was unknown or pending determination,
# which many state departments chose to report separately. In these cases, the county name is “Unknown”
# in the data. For the purposes of our analysis, we will drop this data.

df_covid_combined = df_covid_combined[df_covid_combined['county'] != 'Unknown']

# 3. There are some records where the FIPS code is NA. Let's find out more.

samplenas = df_covid_combined[df_covid_combined['fips'].isna()]

#print(samplenas[samplenas['county']=='New York City'])
#print(samplenas.state.unique())

# There are three counties where the FIPS is NA. Manually adding the FIPS code to these counties.

df_covid_combined.loc[df_covid_combined['county'] == 'Joplin', 'fips'] = '29097'
df_covid_combined.loc[df_covid_combined['county'] == 'Kansas City', 'fips'] = '29095'
df_covid_combined.loc[df_covid_combined['county'] == 'New York City', 'fips'] = '36061'

# Cross checking if there are anymore counties with NA

samplenas2 = df_covid_combined[df_covid_combined['fips'].isna()]
#print(samplenas2.county.unique())

# 4 There are records where the Deaths column is NA. Converting them to 0.

df_covid_combined['deaths'] = df_covid_combined['deaths'].fillna(0)

# 5. Make the date column of type datetime, fips column of type string, cases and deaths as type int.

df_covid_combined['date'] = pd.to_datetime(df_covid_combined['date'])
df_covid_combined['fips'] = df_covid_combined['fips'].astype(int)
df_covid_combined['fips'] = df_covid_combined['fips'].astype(str)
df_covid_combined['cases'] = df_covid_combined['cases'].astype(int)
df_covid_combined['deaths'] = df_covid_combined['deaths'].astype(int)

# 6. Let's extract all months, days and day of the week for later analysis.
df_covid_combined['year'] = df_covid_combined['date'].dt.year.astype(str)
df_covid_combined['month'] = df_covid_combined['date'].dt.month.astype(str)
df_covid_combined['day'] = df_covid_combined['date'].dt.day.astype(str)
df_covid_combined['dayofweek'] = df_covid_combined['date'].dt.day_name()

df_covid_combined.head(4)

Shape of the dataset:(2984844, 6)


,date,county,state,fips,cases,deaths,year,month,day,dayofweek
0,2020-01-21,Snohomish,Washington,53061,1,0,2020,1,21,Tuesday
1,2020-01-22,Snohomish,Washington,53061,1,0,2020,1,22,Wednesday
2,2020-01-23,Snohomish,Washington,53061,1,0,2020,1,23,Thursday
3,2020-01-24,Cook,Illinois,17031,1,0,2020,1,24,Friday


In [ ]:
# Convert datatype of the columns to relevant type
df_covid_combined['month_date'] = pd.to_datetime(df_covid_combined['date']) + pd.offsets.MonthBegin(-1)

covid_case_deaths_agg = df_covid_combined\
                            .groupby(by=['month_date','fips','county','state']) \
                            .agg(sum_cases=pd.NamedAgg(column="cases", aggfunc="sum"),\
                                 max_cases=pd.NamedAgg(column="cases", aggfunc="max"),\
                                 sum_deaths=pd.NamedAgg(column="deaths", aggfunc="sum"),\
                                 max_deaths=pd.NamedAgg(column="deaths", aggfunc="max"))\
                            .reset_index()
covid_case_deaths_agg.head(5)

,month_date,fips,county,state,sum_cases,max_cases,sum_deaths,max_deaths
0,2020-01-01,17031,Cook,Illinois,12,2,0,0
1,2020-01-01,25025,Suffolk,Massachusetts,1,1,0,0
2,2020-01-01,4013,Maricopa,Arizona,7,1,0,0
3,2020-01-01,53061,Snohomish,Washington,12,1,0,0
4,2020-01-01,6037,Los Angeles,California,7,1,0,0


In [ ]:
# 5. Since the US housing market data is at a monthly level, we will resample the COVID-19 data 
# from daily level to monthly level so that they can be easily overlaid and compared.

df_groupby_month = df_covid_combined.groupby([df_covid_combined['county'], df_covid_combined['year'], df_covid_combined['month']])['cases'].sum()
df_groupby_month = df_groupby_month.reset_index()

#df_groupby_month['mom change'] = df_groupby_month.cases.diff()

print(df_groupby_month.head(50))

df_groupby_county_cases = df_covid_combined.groupby('fips')['cases'].sum()
df_groupby_county_deaths = df_covid_combined.groupby('fips')['deaths'].sum()

df_groupby_county_cases



       county  year  month   cases
0   Abbeville  2020      3      32
1   Abbeville  2020      4     430
2   Abbeville  2020      5    1080
3   Abbeville  2020      6    2187
4   Abbeville  2020      7    5959
5   Abbeville  2020      8   11082
6   Abbeville  2020      9   15286
7   Abbeville  2020     10   21806
8   Abbeville  2020     11   26305
9   Abbeville  2020     12   35084
10  Abbeville  2021      1   46891
11  Abbeville  2021      2   58234
12  Abbeville  2021      3   72427
13  Abbeville  2021      4   75231
14  Abbeville  2021      5   79939
15  Abbeville  2021      6   77960
16  Abbeville  2021      7   80836
17  Abbeville  2021      8   86177
18  Abbeville  2021      9  100132
19  Abbeville  2021     10  116342
20  Abbeville  2021     11  116838
21  Abbeville  2021     12  126704
22  Abbeville  2022      1  160983
23  Abbeville  2022      2  181654
24  Abbeville  2022      3  205186
25  Abbeville  2022      4  199209
26  Abbeville  2022      5  206572
27  Abbeville  2022 

fips
0        1151133761
10001      22421254
10003      69212857
10005      29548816
1001        7674855
            ...    
9007       13549768
9009       94378927
9011       25240359
9013       10011097
9015       11746438
Name: cases, Length: 3221, dtype: int64

### COVID-19 Vaccination Data (secondary)

Paragraph

In [1]:
import pandas as pd
import plotly.express as px
pd.options.mode.chained_assignment = None 

In [2]:
# loading COVID vaccination data set 
covid_vaccination_data_temp = pd.read_csv('covid_vaccination_data.csv', low_memory=False)

In [3]:
covid_vaccination_data_temp.columns

Index(['Unnamed: 0', 'date', 'fips', 'mmwr_week', 'recip_county',
       'recip_state', 'completeness_pct', 'administered_dose1_recip',
       'administered_dose1_pop_pct', 'administered_dose1_recip_5plus',
       'administered_dose1_recip_5pluspop_pct',
       'administered_dose1_recip_12plus',
       'administered_dose1_recip_12pluspop_pct',
       'administered_dose1_recip_18plus',
       'administered_dose1_recip_18pluspop_pct',
       'administered_dose1_recip_65plus',
       'administered_dose1_recip_65pluspop_pct', 'series_complete_yes',
       'series_complete_pop_pct', 'series_complete_5plus',
       'series_complete_5pluspop_pct', 'series_complete_5to17',
       'series_complete_5to17pop_pct', 'series_complete_12plus',
       'series_complete_12pluspop_pct', 'series_complete_18plus',
       'series_complete_18pluspop_pct', 'series_complete_65plus',
       'series_complete_65pluspop_pct', 'booster_doses',
       'booster_doses_vax_pct', 'booster_doses_5plus',
       'booster_d

In [4]:
# Keep only relevant columns
covid_vaccination_data = covid_vaccination_data_temp[['date','fips','recip_county','recip_state',\
                            'administered_dose1_pop_pct','metro_status','census2019','census2019_5pluspop', \
                            'census2019_5to17pop', 'census2019_12pluspop','census2019_18pluspop', \
                            'census2019_65pluspop']]
covid_vaccination_data.sample(5)

,date,fips,recip_county,recip_state,administered_dose1_pop_pct,metro_status,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop
727060,2021-11-22T00:00:00.000,21207,Russell County,KY,47.3,Non-metro,17923.0,NaN,NaN,15178.0,13863.0,NaN
1182717,2021-07-06T00:00:00.000,21005,Anderson County,KY,47.4,Non-metro,22747.0,NaN,NaN,19284.0,17402.0,NaN
1795907,2020-12-31T00:00:00.000,46073,Jerauld County,SD,0.0,Non-metro,2013.0,NaN,NaN,1689.0,1539.0,NaN
134819,2022-05-21T00:00:00.000,31093,Howard County,NE,NaN,Metro,6445.0,6041.0,1151.0,5432.0,4890.0,1395.0
493976,2022-02-01T00:00:00.000,19033,Cerro Gordo County,IA,67.2,Non-metro,42450.0,40137.0,NaN,36588.0,33623.0,9484.0


In [5]:
# Convert datatype of the columns to relevant type
covid_vaccination_data['date'] = pd.to_datetime(covid_vaccination_data['date'])
covid_vaccination_data['month_date'] = pd.to_datetime(covid_vaccination_data['date']) + pd.offsets.MonthBegin(-1)

In [6]:
# Impute missing data with appropriate values
covid_vaccination_data['recip_state'] = covid_vaccination_data['recip_state'].fillna('Missing')
covid_vaccination_data['metro_status'] = covid_vaccination_data['metro_status'].fillna('Missing')
covid_vaccination_data['administered_dose1_pop_pct'] = covid_vaccination_data['administered_dose1_pop_pct'].fillna(0)
covid_vaccination_data['census2019'] = covid_vaccination_data['census2019'].fillna(0)
covid_vaccination_data['census2019_5pluspop'] = covid_vaccination_data['census2019_5pluspop'].fillna(0)
covid_vaccination_data['census2019_5to17pop'] = covid_vaccination_data['census2019_5to17pop'].fillna(0)
covid_vaccination_data['census2019_12pluspop'] = covid_vaccination_data['census2019_12pluspop'].fillna(0)
covid_vaccination_data['census2019_18pluspop'] = covid_vaccination_data['census2019_18pluspop'].fillna(0)
covid_vaccination_data['census2019_65pluspop'] = covid_vaccination_data['census2019_65pluspop'].fillna(0)


In [ ]:
#covid_vaccination_data[covid_vaccination_data['county']=='Unknown County']

In [7]:
# Rename column names
covid_vaccination_data = covid_vaccination_data.rename(columns={'recip_county':'county','recip_state':'state'})

# Remove data from 'Unknown County'
covid_vaccination_data = covid_vaccination_data[covid_vaccination_data['county'] !='Unknown County']

covid_vaccination_data.head(5)

,date,fips,county,state,administered_dose1_pop_pct,metro_status,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop,month_date
0,2022-09-28,26039,Crawford County,MI,52.9,Non-metro,14029.0,13327.0,1832.0,12409.0,11495.0,3620.0,2022-09-01
1,2022-09-28,40015,Caddo County,OK,77.8,Non-metro,28762.0,26833.0,5227.0,23993.0,21606.0,4861.0,2022-09-01
2,2022-09-28,17007,Boone County,IL,67.4,Metro,53544.0,50538.0,10131.0,45636.0,40407.0,8642.0,2022-09-01
3,2022-09-28,12055,Highlands County,FL,65.9,Metro,106221.0,101611.0,13261.0,94526.0,88350.0,38289.0,2022-09-01
4,2022-09-28,34029,Ocean County,NJ,66.0,Metro,607186.0,563917.0,103421.0,507517.0,460496.0,138328.0,2022-09-01


In [8]:
covid_vaccination_agg = covid_vaccination_data\
                            .groupby(by=['month_date','fips','county','state','metro_status']) \
                            .last() \
                            .reset_index()

In [ ]:
covid_vaccination_agg.head(5)

,month_date,fips,county,state,metro_status,date,completeness_pct,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop
0,2020-12-01,01001,Autauga County,AL,Metro,2020-12-13,0.0,55869.0,0.0,0.0,47574.0,42904.0,0.0
1,2020-12-01,01003,Baldwin County,AL,Metro,2020-12-13,0.0,223234.0,0.0,0.0,192649.0,175680.0,0.0
2,2020-12-01,01005,Barbour County,AL,Non-metro,2020-12-13,0.0,24686.0,0.0,0.0,21404.0,19604.0,0.0
3,2020-12-01,01007,Bibb County,AL,Metro,2020-12-13,0.0,22394.0,0.0,0.0,19480.0,17837.0,0.0
4,2020-12-01,01009,Blount County,AL,Metro,2020-12-13,0.0,57826.0,0.0,0.0,49234.0,44571.0,0.0


### FIPS code dataset (Secondary)

In [ ]:
fips_code = pd.read_csv('fips_code.csv')
fips_code = fips_code.rename(columns={'Name':'county_key','State':'state_code','FIPS':'fips'})

### Joining all the datasets

Paragraph

In [ ]:
df_groupby_month.head(2)

,county,year,month,cases
0,Abbeville,2020,3,32
1,Abbeville,2020,4,430


In [ ]:
covid_vaccination_agg.head(2)

,month_date,fips,county,state,metro_status,date,completeness_pct,census2019,census2019_5pluspop,census2019_5to17pop,census2019_12pluspop,census2019_18pluspop,census2019_65pluspop
0,2020-12-01,01001,Autauga County,AL,Metro,2020-12-13,0.0,55869.0,0.0,0.0,47574.0,42904.0,0.0
1,2020-12-01,01003,Baldwin County,AL,Metro,2020-12-13,0.0,223234.0,0.0,0.0,192649.0,175680.0,0.0


In [ ]:

fips_counties_with_city = ['Chesapeake, VA', 'Baltimore, MD','St Louis, MO', 'Carson, NV','Alexandria, VA', 'Lynchburg, VA',\
                          'Petersburg, VA','Falls Church, VA','Waynesboro, VA','Portsmouth, VA','Virginia Beach, VA','James, VA',\
                          'Hampton, VA','Harrisonburg, VA','Fredericksburg, VA','Suffolk, VA','Staunton, VA','Salem, VA',\
                          'Martinsville, VA','Manassas Park, VA','Manassas, VA','Poquoson, VA','Lexington, VA','Charlottesville, VA',\
                          'Newport News, VA','Williamsburg, VA','Hopewell, VA','Charles, VA','Winchester, VA','Covington, VA','Norfolk, VA',\
                          'Buena Vista, VA', 'Radford, VA', 'Danville, VA', 'Galax, VA', 'Emporia, VA']

def clean_county(county):
    """
        Input: County value from housing data
        Output: Cleaned county value to match with the format in FIPS data
    """
    
    # Get the county name using spilt function. Some county names have "county" keyword and 
    # others have "," operator. Depending on this extract the county name and the two character state code.
    # Also, remove any leading or trailing spaces
    
    if len(county.split("County"))==1: # county = Emporia, VA
        
        county_value = county.split(",")[0].strip() # county_value = Emporia
        state = county.split(",")[-1].strip() # state = VA
        
    else: # County = Jefferson County, VA
        
        county_value = county.split("County")[0].strip() # county_value = Jefferson
        state = county.split(",")[-1].strip() # state = VA
    
    #remove any period characters in the county names
    county_value = county_value.replace('.','').replace("'",'')
    
    # There are county names like La Moure, De Kalb in FIPS data that is captured as LaMoure and DeKalb
    # Ensure that this is also handled 
    if county_value.startswith('La') or county_value.startswith('De') or county_value.startswith('Du'):
        county_value = re.sub(r"(\w)([A-Z])", r"\1 \2", county_value)
    
    # Remove additional info on county names like 'Parish','City', 'Borough' etc..
    regex = re.compile(r'( Parish| City| Borough)')
    county_value = regex.sub('', county_value)
    
    # Some of the counties in the FIPS dataset have "City" in them and we need to 
    # make these additions to ensure that the format matches with the base FIPS data 
    
    county_state = county_value + ', ' + state
    
    if county_state in fips_counties_with_city:
        county_value = county_value + ' ' + 'City'
    
    # Return the correctly formatted county value
    return(county_value)


In [ ]:
df_housing['county_key'] = df_housing['region'].apply(lambda x: clean_county(x))
df_housing_fips = pd.merge(df_housing, fips_code, how='left', on = ['county_key', 'state_code'], indicator=True)
df_housing_fips['fips'] = df_housing_fips['fips'].astype('str')

### Merge Housing with Vaccination data

In [ ]:
covid_vaccination_agg['fips'] = covid_vaccination_agg['fips'].astype('str')
df_housing_vaccine = pd.merge(df_housing_fips, covid_vaccination_agg, left_on=['fips','period_begin'], right_on=['fips','month_date'], how='left')


### Merge Housing with Vaccination and COVID cases data

In [ ]:
covid_case_deaths_agg['fips'] = covid_case_deaths_agg['fips'].astype('int').astype('str')
df_housing_covid_cases_vaccine = pd.merge(df_housing_vaccine, covid_case_deaths_agg, left_on=['fips','period_begin'], right_on=['fips','month_date'], how='left')

## Analysis

### 📈 Exploratory Data Analysis

We will perform initial investigations on the dataset to discover patterns, spot anomalies, test out hypotheses, and check our assumptions with the help of summary statistics and graphical representations.

We will perform all the four types of EDA: univariate non-graphical, multivariate non-graphical, univariate graphical, and multivariate graphical.

In univariate non-graphical EDA, we will determine the following characteristics of the population distribution for quantitative variables of interest: center, spread, modality, shape, and outliers. In univariate graphical EDA, we will plot histograms and violin plots to visually learn about the dataset.

In multivariate non-graphical EDA, we will explore the relationship between two or more quantitative variables of interest in the dataset by measuring correlation and covariance. In multivariate graphical EDA, we will look to build a grouped box plot that shows the relationship between two or more variables of interest in the dataset.

In [ ]:
# Code block @Ajay
# at data set level
# also at combined data set

### 🏠 🦠 Correlation between housing prices and COVID-19 positivity rate

To understand the correlation between housing market variables and COVID-19 variables, we will employ features like median sale price, median list price, avg_sale_to_list_price, homes_sold, inventory etc. with COVID-19 related features like number of cases, number of deaths, vaccination rate etc.

We will also explore the correlation between month-over-month (m-o-m) housing features like inventory m-o-m, median sale price m-o-m, median list price m-o-m with COVID-19 features also computed at m-o-m like number of cases m-o-m, number of deaths m-o-m, etc.

We plan to do the correlation analysis by different segments like metro vs. non-metro counties, low vs. high-priced housing market areas at county level (only considering top 10 counties (in terms of number of homes sold), etc.

From this analysis, we are expecting to find slight negative correlation between features like inventory and COVID-19 cases but more stronger negative correlation when we analyze the correlation of m-o-m features. This is stemming from the hypothesis that housing sale and inventory features would have a lag effect with COVID-19 features. This analysis will help explain variance in housing inventory m-o-m caused by rise or drop in COVID-19 cases m-o-m.

In [ ]:
# Code block @Varshini

### 🔁 Autocorrelation

To analyze the time series housing sale price data we will construct autocorrelation plots. This will help us understand if there is a periodicity in the data and if patterns and correlations exist. We will construct two autocorrelation plots — one for the pre-COVID-19 time period and one during the COVID-19 time period. We also plan to plot these two charts for the top 10 counties (in terms of number of homes sold).

At an overall level, we hope to see a positive correlation in median sale price of houses with a lag of 52 weeks in pre-COVID-19 times. We expect COVID-19 to break this correlation at an overall level. But at the county level, we still expect to find counties (which have valuable real estate markets) with a positive correlation in median sale price even during the COVID-19 time period suggesting that the annual trends are holding even during COVID-19.

In [ ]:
# Code block @Aparna

### ⏱ Time series analysis

We will extract trends, seasonality, and cyclical patterns from the time series housing sale price data of the top 10 counties using various rolling window moving average approaches. This also has the effect of smoothing the time series and removing noise, allowing us to analyze the time series in novel ways and draw inferences effectively. We expect to observe interesting patterns pre-COVID-19 and post-COVID-19 through the extraction of these components.

We will also attempt to forecast and plot the median house sale price pre-COVID-19 through 2022 and overlay it on the actual trends observed in median house sale price through 2022. This would allow us to compare what the sale prices might have been without the COVID-19 phenomenon having taken place versus what they actually are during the COVID-19 period. While this does not imply causation, we expect this analysis to provide some semblance of intuitive correlation between the median house sale price and the occurrence of the COVID-19 event.

In [ ]:
# Code block

## Visualizations

### Story 1

To get started with the exploration of housing sale price data and impact of COVID-19 on sale price, we will plot the median sale price data and overlay with COVID-19 cases as line charts over time to identify any patterns or spot anomalies. We will also research the COVID-19 news timeline to highlight any significant events on this chart.

In [ ]:
# Code block

### Story 2

Next we will employ heatmap to explore how housing sale prices have fluctuated over time for various states in the U.S. We would encode the median sale price with color, month-year would be represented along the x-axis and each state in the U.S. would be represented along the y-axis. 

We will also use heatmap to visualize the correlation between various housing price features like median sale price, median sale price yoy (year over year change), median sale price mom (month over month change), median list price, number of homes sold, etc. and COVID-19 features like number of cases, number of deaths, vaccination rate, etc.

In [ ]:
# Code block

## Summary of insights

Paragraph here

## Conclusion & next steps

Paragraph here

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4be7b53e-a34b-46bd-9365-cf5d239dbb44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>